<a href="https://colab.research.google.com/github/gireek/karpathy_lectures/blob/main/makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# why bgram not good - context only for 2 chars

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!git clone https://github.com/karpathy/makemore

Cloning into 'makemore'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 64 (delta 16), reused 13 (delta 13), pack-reused 47
Unpacking objects: 100% (64/64), 123.34 KiB | 1.20 MiB/s, done.


In [3]:
names_path = "./makemore/names.txt"
words = open(names_path, 'r').read().splitlines()

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# didt think about ...-> e (bcz all empty before was an important case too)

block_size = 3

def build_dataset(words):
  X, Y = [], []
  for w in words:

    # print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      # print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  return X, Y


X, Y = build_dataset(words[:5])

In [66]:
num_dimensions = 2

C = torch.randn(27, 2)

C[5]

tensor([-0.8770, -0.4380])

Pytorch indexing is amzaing!!!

In [67]:
C[torch.tensor([5, 5, 5])]

tensor([[-0.8770, -0.4380],
        [-0.8770, -0.4380],
        [-0.8770, -0.4380]])

In [68]:
C[X].shape, X.shape

(torch.Size([32, 3, 2]), torch.Size([32, 3]))

In [69]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

C[X] @ W1

RuntimeError: ignored

In [70]:
emb = C[X].reshape((C[X].shape[0] , -1))

In [32]:
# ineefficient since creates new memory
torch.cat(torch.unbind(C[X], 1), 1).shape

torch.Size([32, 6])

In [38]:
C[X].view((32,6)).shape

torch.Size([32, 6])

In [37]:
# how its sotred internally... u can move it around using view
# C[X].storage()

In [44]:
(emb @ W1).shape

torch.Size([32, 100])

In [71]:
# to add bias per row we want to add [1, num_col] or [num_col] tensor. Latter works bcz automatically converted to former
# to add bias per col we want to add [num_rows, 1] tensor

# therefore

h = torch.tanh((emb @ W1) + b1)
h

tensor([[-0.9997,  0.9997, -1.0000,  ...,  0.1906,  0.9926, -1.0000],
        [-0.8409,  0.9995, -0.9999,  ..., -0.9064,  0.5344, -1.0000],
        [-0.9951,  0.9967, -0.9991,  ...,  0.9905,  0.9902, -1.0000],
        ...,
        [-0.9972,  0.9999, -0.9999,  ...,  0.8426,  0.9722, -0.9904],
        [ 0.9985, -0.2426,  0.9925,  ...,  0.8610, -0.7979, -0.9992],
        [ 0.9617, -0.9551,  0.9898,  ..., -0.7819,  0.9241, -0.1132]])

In [72]:
W2 = torch.randn(100 , 27)
b2 = torch.randn(27)

logits = (h @ W2) + b2
print(logits.shape)

torch.Size([32, 27])


In [78]:
output = torch.softmax(logits, axis = 1)
print(sum(output[0]))
print(-output[torch.arange(32), Y].log().mean())

tensor(1.0000)
tensor(16.2188)


In [79]:
counts = logits.exp()
prob = counts/ counts.sum(1, keepdim=True)
print(sum(prob[0]))
print(-prob[torch.arange(32), Y].log().mean())

tensor(1.0000)


tensor(16.2188)

In [85]:
# output = torch.softmax(logits, axis = 1)
# loss = -output[torch.arange(32), Y].log().mean()

# can be replaced by

print(F.cross_entropy(logits, Y))

## why?
## pytroch optimimzation
## backward pass efficient
## numerically stable

tensor(16.2188)


In [87]:
logits = torch.tensor([-100, 23,2,100])
counts = logits.exp()
probs= counts / counts.sum()
counts, probs

(tensor([3.7835e-44, 9.7448e+09, 7.3891e+00,        inf]),
 tensor([0., 0., 0., nan]))

make it stable as follows

In [88]:
logits = torch.tensor([-100, 23,2,100]) - 100
counts = logits.exp()
probs= counts / counts.sum()
counts, probs

(tensor([0.0000e+00, 3.6251e-34, 2.7465e-43, 1.0000e+00]),
 tensor([0.0000e+00, 3.6251e-34, 2.7465e-43, 1.0000e+00]))

NEATLY

In [6]:
 X, Y = build_dataset(words)
 print(X.shape, Y.shape)

torch.Size([228146, 3]) torch.Size([228146])


In [7]:
X = X.cuda()
Y = Y.cuda()

In [8]:
C = torch.randn(27, 2).cuda()
W1 = torch.randn((6, 100)).cuda()
b1 = torch.randn(100).cuda()
W2 = torch.randn(100 , 27).cuda()
b2 = torch.randn(27).cuda()

params = [C, W1, b1, W2, b2]

for param in params:
  param.requires_grad = True

In [9]:
lre = torch.linspace(-3, -1, 1000)
lrs = 10**lre

In [10]:
lrs[:10], lrs[-1]

(tensor([0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
         0.0010]),
 tensor(0.1000))

In [14]:
num_epochs = 1000

for i in range(num_epochs):
  emb = C[X].reshape((C[X].shape[0] , -1))
  h = torch.tanh((emb @ W1) + b1)
  logits = (h @ W2) + b2
  loss = F.cross_entropy(logits, Y)
  print(loss)

  # backward
  for param in params:
    param.grad = None

  loss.backward()

  lr = 0.07
  for param in params:
    param.data -= lr * param.grad


tensor(2.6342, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6341, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6339, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6338, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6336, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6334, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6333, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6331, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6330, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6328, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6327, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6325, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6323, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6322, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6320, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6319, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6317, device='cuda:0', grad_fn=